# Importing Libaries

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geocoder
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
!pip install folium
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Libraries imported.


# Getting the Data

Build a dataframe of neighborhoods in Hyderabad, India by web scraping the data from Wikipedia page
Get the geographical coordinates of the neighborhoods by Python Geocoder package
Obtain the venue data for the neighborhoods from Foursquare API
Explore and cluster the neighbourhoods
Select the best cluster to open a new shopping mall

# Business Problem

This project is mainly focused on geospatial analysis of the Hyderabad City to understand which would be the best place to open a new mall

In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

In [5]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [6]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,List of neighbourhoods in Hyderabad
1,A. C. Guards
2,A. S. Rao Nagar
3,Abhyudaya Nagar
4,Abids


In [7]:
# print the number of rows of the dataframe
kl_df.shape

(200, 1)

In [8]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [10]:
coords

[[17.320740000000058, 78.46563000000003],
 [17.39267335813635, 78.45633291385879],
 [17.44240132263719, 78.44095428065715],
 [17.337650000000053, 78.56414000000007],
 [17.389810000000068, 78.47659000000004],
 [17.235820000000047, 78.54134000000005],
 [17.410610000000077, 78.51512000000008],
 [17.377490000000023, 78.48004000000003],
 [17.389171673526732, 78.46385520726426],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.53541000000007, 78.54424000000006],
 [17.385820000000024, 78.51836000000003],
 [17.53349000000003, 78.32528000000008],
 [17.435350000000028, 78.44863000000004],
 [17.458180000000027, 78.53886000000006],
 [17.40784000000002, 78.49150000000003],
 [17.38516000000004, 78.44736000000006],
 [17.369170000000054, 78.43683000000004],
 [17.40709000000004, 78.50232000000005],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.399290000000065, 78.49965000000003],
 [

In [11]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [12]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [13]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head()

(200, 3)


,Neighborhood,Latitude,Longitude
0,List of neighbourhoods in Hyderabad,17.320740,78.465630
1,A. C. Guards,17.392673,78.456333
2,A. S. Rao Nagar,17.442401,78.440954
3,Abhyudaya Nagar,17.337650,78.564140
4,Abids,17.389810,78.476590


In [14]:
# get the coordinates of Hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.360589, 78.4740613.


In [15]:
# create map of Hyderabad using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [17]:
# save the map as HTML file
map_kl.save('map_kl.html')

# Use the foursquare API to explore the neighbourhoods

In [18]:
CLIENT_ID = 'RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K' # your Foursquare ID
CLIENT_SECRET = 'TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RJ3CLZG4JCDCVREY5S2R1WFPNX0A03MYJRD4CTPAWWBQJW2K
CLIENT_SECRET:TJ4NPL40BJWU4A51PSHNXVK2ADD2FZRPANQWRNONGGDAMCYN


In [19]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [20]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(6492, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,List of neighbourhoods in Hyderabad,17.320740,78.465630,Taj Falaknuma Palace,17.330118,78.467460,Resort
1,List of neighbourhoods in Hyderabad,17.320740,78.465630,Jade Terrace,17.330542,78.467510,Breakfast Spot
2,List of neighbourhoods in Hyderabad,17.320740,78.465630,Celesté,17.330125,78.467442,Italian Restaurant
3,List of neighbourhoods in Hyderabad,17.320740,78.465630,Yum Yum Tree Arabian,17.309767,78.475790,Mediterranean Restaurant
4,List of neighbourhoods in Hyderabad,17.320740,78.465630,Adaa,17.330192,78.467430,Indian Restaurant
5,List of neighbourhoods in Hyderabad,17.320740,78.465630,Falaknuma station,17.331396,78.473738,Train Station
6,List of neighbourhoods in Hyderabad,17.320740,78.465630,Yum Yum Tree,17.313641,78.481913,Falafel Restaurant
7,List of neighbourhoods in Hyderabad,17.320740,78.465630,Arabian Al-Khaleej Restuarent,17.312814,78.477273,Middle Eastern Restaurant
8,A. C. Guards,17.392673,78.456333,Cafe Niloufer & Bakers,17.399715,78.462881,Café
9,A. C. Guards,17.392673,78.456333,Subhan Bakery,17.392412,78.464712,Bakery


# Lets check how many venues were returned for each neighbourhood

In [21]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. C. Guards,49,49,49,49,49,49
A. S. Rao Nagar,67,67,67,67,67,67
Abhyudaya Nagar,9,9,9,9,9,9
Abids,79,79,79,79,79,79
Adikmet,23,23,23,23,23,23
Afzal Gunj,50,50,50,50,50,50
Aghapura,64,64,64,64,64,64
"Aliabad, Hyderabad",9,9,9,9,9,9
Alijah Kotla,16,16,16,16,16,16


In [ ]:
#venues
#venues_df[venues_df['Neighborhood']=='Sikh Village'].groupby

# Let's find out how many unique categories can be curated from all the returned venues

In [23]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 179 uniques categories.


In [24]:
venues_df['VenueCategory'].unique()[:50] #displays all the 174 category names

array(['Resort', 'Breakfast Spot', 'Italian Restaurant',
       'Mediterranean Restaurant', 'Indian Restaurant', 'Train Station',
       'Falafel Restaurant', 'Middle Eastern Restaurant', 'Café',
       'Bakery', 'Hyderabadi Restaurant', 'Juice Bar', 'Lounge',
       'South Indian Restaurant', 'Park', 'Bistro', 'Ice Cream Shop',
       'Science Museum', 'Snack Place', 'Performing Arts Venue',
       'Vegetarian / Vegan Restaurant', 'Hotel Bar', 'Pizza Place',
       'Stadium', 'Hotel', 'Fast Food Restaurant', 'Mobile Phone Shop',
       'Coffee Shop', 'Fried Chicken Joint', 'Department Store',
       'Hookah Bar', 'Electronics Store', 'Clothing Store', 'Church',
       'Diner', 'Chinese Restaurant', 'Pub', 'Dessert Shop', 'Bar',
       'Concert Hall', 'Rajasthani Restaurant', 'Sandwich Place',
       'Punjabi Restaurant', 'Deli / Bodega', 'Gym', 'Food',
       'Fruit & Vegetable Store', 'Bus Station', "Women's Store",
       'Movie Theater'], dtype=object)

In [25]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

# Analyze each neighbourhood

In [26]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head(20)

(6492, 180)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Lawyer,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,List of neighbourhoods in Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,List of neighbourhoods in Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,List of neighbourhoods in Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,List of neighbourhoods in Hyderabad,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).sum().reset_index()
#df.group/by("state")["last_name"].count()

#kl_onehot.head()
print(kl_grouped.shape)
kl_grouped

(198, 180)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Burger Joint,Bus Station,Business Service,Cable Car,Cafeteria,Café,Candy Store,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Lawyer,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Neighborhood,New American Restaurant,Night Market,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Punjabi Restaurant,Rajasthani Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. C. Guards,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,1,0,0,2,0,2,0,9,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,3,0,0,0,5,0,0,0,0,3,0,1,0,1,0,1,0,0,0,0,0,0,2,2,2,0,2,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,2,0,0,0,0,0,2,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,1,1,1,0,0,0,0,0,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,4,0,1,0
2,Abhyudaya Nagar,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abids,0,0,0,0,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,2,0,0,0,3,0,0,1,0,4,1,1,0,2,0,0,0,0,0,0,0,0,0,1,3,0,1,0,0,1,0,0,0,1,4,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,1,0,0,0,0,6,0,9,0,1,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,3,0,0,0,1,0,0,1,1,1,2,0,2,0,0,0

In [ ]:
#a=kl_grouped['Neighborhoods'][0]


In [28]:
len((kl_grouped[kl_grouped["Shopping Mall"] > 0]))  #So there are 79 shopping malls in hyderabad(which it very high)
#So now we want to select a good location where the no of shopping malls are less so that our chances of setting up a shopping mall at that location  should be good

71

# Create a dataframe for shopping mall data only

In [29]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [30]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,A. C. Guards,0
1,A. S. Rao Nagar,0
2,Abhyudaya Nagar,0
3,Abids,1
4,Adikmet,0


# Now cluster the neighbourhoods

Run k-means to cluster the neighborhoods in Hyderabad into 3 clusters.

In [31]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

<ipython-input-31-59131dde4041>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  kl_clustering = kl_mall.drop(["Neighborhoods"], 1)


array([1, 1, 1, 0, 1, 2, 0, 1, 0, 1], dtype=int32)

In [32]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [33]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head(10)

,Neighborhood,Shopping Mall,Cluster Labels
0,A. C. Guards,0,1
1,A. S. Rao Nagar,0,1
2,Abhyudaya Nagar,0,1
3,Abids,1,0
4,Adikmet,0,1
5,Afzal Gunj,2,2
6,Aghapura,1,0
7,"Aliabad, Hyderabad",0,1
8,Alijah Kotla,1,0
9,Allwyn Colony,0,1


In [34]:
#Add latitude and longitude values by using the join operation(the new dataframe with the old dataframe containing the latitude and longitude values)
#kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

#print(kl_merged.shape)
kl_merged['Latitude'] = df_coords['Latitude']
kl_merged['Longitude'] = df_coords['Longitude']
kl_merged.head() # check the last columns!

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. C. Guards,0,1,17.320740,78.465630
1,A. S. Rao Nagar,0,1,17.392673,78.456333
2,Abhyudaya Nagar,0,1,17.442401,78.440954
3,Abids,1,0,17.337650,78.564140
4,Adikmet,0,1,17.389810,78.476590


In [35]:
# sorting the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(198, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
171,Quthbullapur,1,0,17.368160,78.458180
129,Masab Tank,1,0,17.401390,78.391660
172,RTC X Roads,1,0,17.383174,78.477501
39,Chaderghat,1,0,17.452700,78.412560
170,Putlibowli,1,0,17.426160,78.452100
94,Khajaguda,1,0,17.379070,78.436680
43,Chatta Bazaar,1,0,17.369180,78.495570
166,Pisal Banda,1,0,17.442320,78.496170
165,Pet Basheerabad,1,0,17.526770,78.252340
46,Chikkadpally,1,0,17.437070,78.606840


In [36]:
kl_merged['Shopping Mall'].max()

2

# Now we visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examine clusters

Cluster 0

In [39]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 0])# -132 neighbourhoods/places in this cluster 0

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
171,Quthbullapur,1,0,17.368160,78.458180
129,Masab Tank,1,0,17.401390,78.391660
172,RTC X Roads,1,0,17.383174,78.477501
39,Chaderghat,1,0,17.452700,78.412560
170,Putlibowli,1,0,17.426160,78.452100
94,Khajaguda,1,0,17.379070,78.436680
43,Chatta Bazaar,1,0,17.369180,78.495570
166,Pisal Banda,1,0,17.442320,78.496170
165,Pet Basheerabad,1,0,17.526770,78.252340
46,Chikkadpally,1,0,17.437070,78.606840


Cluster 1

In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 1])
#There are 51 neighbourhoods/places in cluster 1 which is the highest among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
136,Miyapur,0,1,17.438858,78.480429
138,Moghalpura,0,1,17.421010,78.582470
116,Madannapet,0,1,17.494050,78.515130
117,Madeenaguda,0,1,17.525910,78.376330
140,Moosarambagh,0,1,17.358810,78.477200
119,"Madina building, Hyderabad",0,1,17.495350,78.333350
118,Madhapur,0,1,17.357880,78.501700
128,Marredpally,0,1,17.391860,78.459050
125,Mallapur,0,1,17.447370,78.535200
135,Mir Alam Tank,0,1,17.427740,78.528920


Cluster 2

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2] 
#len(kl_merged.loc[kl_merged['Cluster Labels'] == 2] ) #-There are only 9 shopping malls in the cluster which is the least among the 3 clusters

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
5,Afzal Gunj,2,2,17.235820,78.541340
106,Lab quarters,2,2,17.481130,78.583700
13,Ameerpet,2,2,17.385820,78.518360
120,Maharajgunj,2,2,17.456960,78.390050
153,Narayanguda,2,2,17.413990,78.345800
174,Raj Bhavan Road,2,2,17.425120,78.395190
173,Raidurg,2,2,17.505360,78.467480
137,Moazzam Jahi Market,2,2,17.364481,78.456767
168,Punjagutta,2,2,17.394880,78.470750
84,Jubilee Hills,2,2,17.354420,78.432550


# Conclusion

A good number of shopping malls are concentrated in the central area of Hyderabad city, with the highest number in cluster 2 and moderate number in cluster 1. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighbourhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighbourhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighbourhoods in cluster 2 which already have a high concentration of shopping malls and suffering from intense competition.


